In [1]:
import AdventOfCode
import qualified Data.Vector as V

In [2]:
input <- dayString 5

instructions = read ("[" ++ input ++ "]") :: [Int]

In [3]:
data Opcode = Opcode
    { opcodeCode  :: Int
    , opcodeMode1 :: Bool
    , opcodeMode2 :: Bool
    , opcodeMode3 :: Bool
    } deriving (Eq, Show)

In [4]:
parseCode :: Int -> Opcode
parseCode i = let
    m3   = i `div` 10000
    m2   = (i - m3*10000) `div` 1000
    m1   = (i - m3*10000 - m2*1000) `div` 100
    code = i `rem` 100
    in Opcode code (m1==1) (m2==1) (m3==1)

parseCode 10102

Opcode {opcodeCode = 2, opcodeMode1 = True, opcodeMode2 = False, opcodeMode3 = True}

In [5]:
step :: V.Vector Int -> Int -> IO (V.Vector Int, Int)
step program index = do
    let opcode = parseCode (program V.! index)
    case opcodeCode opcode of
        1 -> let
            a = if opcodeMode1 opcode then (program V.! (index+1)) else program V.! (program V.! (index+1))
            b = if opcodeMode2 opcode then (program V.! (index+2)) else program V.! (program V.! (index+2))
            o = program V.! (index+3)
            p = program V.// [(o,a+b)]
            in pure (p,index+4)
        2 -> let
            a = if opcodeMode1 opcode then (program V.! (index+1)) else program V.! (program V.! (index+1))
            b = if opcodeMode2 opcode then (program V.! (index+2)) else program V.! (program V.! (index+2))
            o = program V.! (index+3)
            p = program V.// [(o,a*b)]
            in pure (p,index+4)
        3 -> do
            v <- read <$> getLine
            let o = program V.! (index+1)
            let p = program V.// [(o, v)]
            pure (p, index+2)
        4 -> do
            let a = if opcodeMode1 opcode then (program V.! (index+1)) else program V.! (program V.! (index+1))
            print a
            pure (program, index+2)
        99 -> pure (program, index)

Line 6: Reduce duplication
Found:
a = if opcodeMode1 opcode then (program V.! (index + 1)) else
      program V.! (program V.! (index + 1))
b = if opcodeMode2 opcode then (program V.! (index + 2)) else
      program V.! (program V.! (index + 2))
o = program V.! (index + 3)

Why not:
Combine with -:12:13Line 6: Redundant bracket
Found:
if opcodeMode1 opcode then (program V.! (index + 1)) else
  program V.! (program V.! (index + 1))
Why not:
if opcodeMode1 opcode then program V.! (index + 1) else
  program V.! (program V.! (index + 1))Line 7: Redundant bracket
Found:
if opcodeMode2 opcode then (program V.! (index + 2)) else
  program V.! (program V.! (index + 2))
Why not:
if opcodeMode2 opcode then program V.! (index + 2) else
  program V.! (program V.! (index + 2))Line 12: Redundant bracket
Found:
if opcodeMode1 opcode then (program V.! (index + 1)) else
  program V.! (program V.! (index + 1))
Why not:
if opcodeMode1 opcode then program V.! (index + 1) else
  program V.! (program V.! (index + 1))Line 13: Redundant bracket
Found:
if opcodeMode2 opcode then (program V.! (index + 2)) else
  program V.! (program V.! (index + 2))
Why not:
if opcodeMode2 opcode then program V.! (index + 2) else
  program V.! (program V.! (index + 2))Line 23: Redundant bracket
Found:
if opcodeMode1 opcode then (program V.! (index + 1)) else
  program V.! (program V.! (index + 1))
Why not:
if opcodeMode1 opcode then program V.! (index + 1) else
  program V.! (program V.! (index + 1))

In [6]:
step (V.fromList instructions) 0

([3,225,1,225,6,6,1100,1,238,225,104,0,2,136,183,224,101,-5304,224,224,4,224,1002,223,8,223,1001,224,6,224,1,224,223,223,1101,72,47,225,1101,59,55,225,1101,46,75,225,1101,49,15,224,101,-64,224,224,4,224,1002,223,8,223,1001,224,5,224,1,224,223,223,102,9,210,224,1001,224,-270,224,4,224,1002,223,8,223,1001,224,2,224,1,223,224,223,101,14,35,224,101,-86,224,224,4,224,1002,223,8,223,101,4,224,224,1,224,223,223,1102,40,74,224,1001,224,-2960,224,4,224,1002,223,8,223,101,5,224,224,1,224,223,223,1101,10,78,225,1001,39,90,224,1001,224,-149,224,4,224,102,8,223,223,1001,224,4,224,1,223,224,223,1002,217,50,224,1001,224,-1650,224,4,224,1002,223,8,223,1001,224,7,224,1,224,223,223,1102,68,8,225,1,43,214,224,1001,224,-126,224,4,224,102,8,223,223,101,3,224,224,1,224,223,223,1102,88,30,225,1102,18,80,225,1102,33,28,225,4,223,99,0,0,1,677,0,0,0,0,0,0,0,0,0,0,0,1105,0,99999,1105,227,247,1105,1,99999,1005,227,99999,1005,0,256,1105,1,99999,1106,227,99999,1106,0,265,1105,1,99999,1006,0,99999,1006,227,274,1105,

In [7]:
loop program index = do
    (program', index') <- step program index
    if index == index' then pure program else loop program' index'

loop (V.fromList instructions) 0

0
0
0
0
0
0
0
0
0
13978427
[3,225,1,225,6,6,1101,1,238,225,104,0,2,136,183,224,101,-5304,224,224,4,224,1002,223,8,223,1001,224,6,224,1,224,223,223,1101,72,47,225,1101,59,55,225,1101,46,75,225,1101,49,15,224,101,-64,224,224,4,224,1002,223,8,223,1001,224,5,224,1,224,223,223,102,9,210,224,1001,224,-270,224,4,224,1002,223,8,223,1001,224,2,224,1,223,224,223,101,14,35,224,101,-86,224,224,4,224,1002,223,8,223,101,4,224,224,1,224,223,223,1102,40,74,224,1001,224,-2960,224,4,224,1002,223,8,223,101,5,224,224,1,224,223,223,1101,10,78,225,1001,39,90,224,1001,224,-149,224,4,224,102,8,223,223,1001,224,4,224,1,223,224,223,1002,217,50,224,1001,224,-1650,224,4,224,1002,223,8,223,1001,224,7,224,1,224,223,223,1102,68,8,225,1,43,214,224,1001,224,-126,224,4,224,102,8,223,223,101,3,224,224,1,224,223,223,1102,88,30,225,1102,18,80,225,1102,33,28,225,4,223,99,13978427,3,924,677,0,0,0,0,0,0,0,0,0,0,0,1105,0,99999,1105,227,247,1105,1,99999,1005,227,99999,1005,0,256,1105,1,99999,1106,227,99999,1106,0,265,1105,1,99

In [16]:
import Debug.Trace

step' :: V.Vector Int -> Int -> IO (V.Vector Int, Int)
step' program index = do
    let opcode = parseCode (program V.! index)
    case opcodeCode opcode of
        1 -> let
            a = if opcodeMode1 opcode then (program V.! (index+1)) else program V.! (program V.! (index+1))
            b = if opcodeMode2 opcode then (program V.! (index+2)) else program V.! (program V.! (index+2))
            o = program V.! (index+3)
            p = program V.// [(o,a+b)]
            in pure (p,index+4)
        2 -> let
            a = if opcodeMode1 opcode then (program V.! (index+1)) else program V.! (program V.! (index+1))
            b = if opcodeMode2 opcode then (program V.! (index+2)) else program V.! (program V.! (index+2))
            o = program V.! (index+3)
            p = program V.// [(o,a*b)]
            in pure (p,index+4)
        3 -> do
            v <- read <$> getLine
            let o = program V.! (index+1)
            let p = program V.// [(o, v)]
            pure (p, index+2)
        4 -> do
            let a = if opcodeMode1 opcode then (program V.! (index+1)) else program V.! (program V.! (index+1))
            traceIO $ show a
            print a
            pure (program, index+2)
        5 -> let
            a = if opcodeMode1 opcode then (program V.! (index+1)) else program V.! (program V.! (index+1))
            b = if opcodeMode2 opcode then (program V.! (index+2)) else program V.! (program V.! (index+2))
            in if a/=0 then pure (program, b) else pure (program, index+3)
        6 -> let
            a = if opcodeMode1 opcode then (program V.! (index+1)) else program V.! (program V.! (index+1))
            b = if opcodeMode2 opcode then (program V.! (index+2)) else program V.! (program V.! (index+2))
            in if a==0 then pure (program, b) else pure (program, index+3)
        7 -> let
            a = if opcodeMode1 opcode then (program V.! (index+1)) else program V.! (program V.! (index+1))
            b = if opcodeMode2 opcode then (program V.! (index+2)) else program V.! (program V.! (index+2))
            o = program V.! (index+3)
            in if a < b then pure (program V.// [(o,1)], index+4) else pure (program V.// [(o, 0)], index+4)
        8 -> let
            a = if opcodeMode1 opcode then (program V.! (index+1)) else program V.! (program V.! (index+1))
            b = if opcodeMode2 opcode then (program V.! (index+2)) else program V.! (program V.! (index+2))
            o = program V.! (index+3)
            in if a == b then pure (program V.// [(o,1)], index+4) else pure (program V.// [(o, 0)], index+4)
        99 -> pure (program, index)

Line 8: Reduce duplication
Found:
a = if opcodeMode1 opcode then (program V.! (index + 1)) else
      program V.! (program V.! (index + 1))
b = if opcodeMode2 opcode then (program V.! (index + 2)) else
      program V.! (program V.! (index + 2))
o = program V.! (index + 3)

Why not:
Combine with -:14:13Line 8: Reduce duplication
Found:
a = if opcodeMode1 opcode then (program V.! (index + 1)) else
      program V.! (program V.! (index + 1))
b = if opcodeMode2 opcode then (program V.! (index + 2)) else
      program V.! (program V.! (index + 2))
o = program V.! (index + 3)

Why not:
Combine with -:38:13Line 8: Reduce duplication
Found:
a = if opcodeMode1 opcode then (program V.! (index + 1)) else
      program V.! (program V.! (index + 1))
b = if opcodeMode2 opcode then (program V.! (index + 2)) else
      program V.! (program V.! (index + 2))
o = program V.! (index + 3)

Why not:
Combine with -:43:13Line 8: Redundant bracket
Found:
if opcodeMode1 opcode then (program V.! (index + 1)) else
  program V.! (program V.! (index + 1))
Why not:
if opcodeMode1 opcode then program V.! (index + 1) else
  program V.! (program V.! (index + 1))Line 9: Redundant bracket
Found:
if opcodeMode2 opcode then (program V.! (index + 2)) else
  program V.! (program V.! (index + 2))
Why not:
if opcodeMode2 opcode then program V.! (index + 2) else
  program V.! (program V.! (index + 2))Line 14: Redundant bracket
Found:
if opcodeMode1 opcode then (program V.! (index + 1)) else
  program V.! (program V.! (index + 1))
Why not:
if opcodeMode1 opcode then program V.! (index + 1) else
  program V.! (program V.! (index + 1))Line 15: Redundant bracket
Found:
if opcodeMode2 opcode then (program V.! (index + 2)) else
  program V.! (program V.! (index + 2))
Why not:
if opcodeMode2 opcode then program V.! (index + 2) else
  program V.! (program V.! (index + 2))Line 25: Redundant bracket
Found:
if opcodeMode1 opcode then (program V.! (index + 1)) else
  program V.! (program V.! (index + 1))
Why not:
if opcodeMode1 opcode then program V.! (index + 1) else
  program V.! (program V.! (index + 1))Line 30: Redundant bracket
Found:
if opcodeMode1 opcode then (program V.! (index + 1)) else
  program V.! (program V.! (index + 1))
Why not:
if opcodeMode1 opcode then program V.! (index + 1) else
  program V.! (program V.! (index + 1))Line 31: Redundant bracket
Found:
if opcodeMode2 opcode then (program V.! (index + 2)) else
  program V.! (program V.! (index + 2))
Why not:
if opcodeMode2 opcode then program V.! (index + 2) else
  program V.! (program V.! (index + 2))Line 34: Redundant bracket
Found:
if opcodeMode1 opcode then (program V.! (index + 1)) else
  program V.! (program V.! (index + 1))
Why not:
if opcodeMode1 opcode then program V.! (index + 1) else
  program V.! (program V.! (index + 1))Line 35: Redundant bracket
Found:
if opcodeMode2 opcode then (program V.! (index + 2)) else
  program V.! (program V.! (index + 2))
Why not:
if opcodeMode2 opcode then program V.! (index + 2) else
  program V.! (program V.! (index + 2))Line 38: Redundant bracket
Found:
if opcodeMode1 opcode then (program V.! (index + 1)) else
  program V.! (program V.! (index + 1))
Why not:
if opcodeMode1 opcode then program V.! (index + 1) else
  program V.! (program V.! (index + 1))Line 39: Redundant bracket
Found:
if opcodeMode2 opcode then (program V.! (index + 2)) else
  program V.! (program V.! (index + 2))
Why not:
if opcodeMode2 opcode then program V.! (index + 2) else
  program V.! (program V.! (index + 2))Line 43: Redundant bracket
Found:
if opcodeMode1 opcode then (program V.! (index + 1)) else
  program V.! (program V.! (index + 1))
Why not:
if opcodeMode1 opcode then program V.! (index + 1) else
  program V.! (program V.! (index + 1))Line 44: Redundant bracket
Found:
if opcodeMode2 opcode then (program V.! (index + 2)) else
  program V.! (program V.! (index + 2))
Why not:
if opcodeMode2 opcode then program V.! (index + 2) else
  program V.! (program V.! (index + 2))

In [17]:
loop' program index = do
    (program', index') <- step' program index
    if index == index' then pure program else loop' program' index'

In [18]:
loop' (V.fromList [3,21,1008,21,8,20,1005,20,22,107,8,21,20,1006,20,31,1106,0,36,98,0,0,1002,21,125,20,4,20,1105,1,46,104,999,1105,1,46,1101,1000,1,20,4,20,1105,1,46,98,99]) 0

999
999
[3,21,1008,21,8,20,1005,20,22,107,8,21,20,1006,20,31,1106,0,36,98,0,5,1002,21,125,20,4,20,1105,1,46,104,999,1105,1,46,1101,1000,1,20,4,20,1105,1,46,98,99]

In [19]:
loop' (V.fromList instructions) 0

11189491
11189491
[314,225,1,225,6,6,1105,1,238,225,104,0,2,136,183,224,101,-5304,224,224,4,224,1002,223,8,223,1001,224,6,224,1,224,223,223,1101,72,47,225,1101,59,55,225,1101,46,75,225,1101,49,15,224,101,-64,224,224,4,224,1002,223,8,223,1001,224,5,224,1,224,223,223,102,9,210,224,1001,224,-270,224,4,224,1002,223,8,223,1001,224,2,224,1,223,224,223,101,14,35,224,101,-86,224,224,4,224,1002,223,8,223,101,4,224,224,1,224,223,223,1102,40,74,224,1001,224,-2960,224,4,224,1002,223,8,223,101,5,224,224,1,224,223,223,1101,10,78,225,1001,39,90,224,1001,224,-149,224,4,224,102,8,223,223,1001,224,4,224,1,223,224,223,1002,217,50,224,1001,224,-1650,224,4,224,1002,223,8,223,1001,224,7,224,1,224,223,223,1102,68,8,225,1,43,214,224,1001,224,-126,224,4,224,102,8,223,223,101,3,224,224,1,224,223,223,1102,88,30,225,1102,18,80,225,1102,33,28,225,4,223,99,11189491,0,20,677,0,0,0,0,0,0,0,0,0,0,0,1105,0,99999,1105,227,247,1105,1,99999,1005,227,99999,1005,0,256,1105,1,99999,1106,227,99999,1106,0,265,1105,1,99999,1006